In [147]:
import pandas as pd
import requests
import urllib3
import ssl
import io

In [148]:
all_countries = ['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
                 'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
                 'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore',
                 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland',
                 'New Zealand', 'Austria', 'Portugal', 'Eurozone',
                 'China', 'Taiwan', 'India', 'Korea', 'Brasil',
                 'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
                 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye',
                 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru',
                 'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia']

In [149]:
all_countries_in_oecd = ['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
                         'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
                         'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway',
                         'Israel', 'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone',
                         'China', 'India', 'Korea', 'Brasil', 'Saudi Arabia', 'South Africa',
                         'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece',
                         'Hungary', 'Czechia', 'Colombia', 'Argentina', 'Russia']

In [150]:
def format_oecd_data(df):
    df_formatted = pd.DataFrame(index=df['TIME'].unique())

    for country in df['Country'].unique():
        df_formatted[country] = df[df['Country'] == country]['Value']

    df_formatted.rename(columns={
        'Czech Republic': 'Czechia',
        'Türkiye': 'Turkiye',
        "China (People's Republic of)": 'China',
        'Euro area (19 countries)': 'Eurozone'
    }, inplace=True)

    df_formatted = df_formatted.reindex(columns=all_countries_in_oecd)
    return df_formatted

In [151]:
def check_data_coverage(df):
    available_countries = df.columns[~df.isna().all()]
    print('Available countries:')
    print(available_countries)
    print()

    no_data = df.columns[df.isna().all()]
    print('No data:')
    print(no_data)
    print()

    print('Missing first:')
    for country in df.columns:
        first_valid = df[country].first_valid_index()
        if first_valid != None and (not first_valid.startswith('1999')):
            print(country, first_valid)
    print()

    print('Missing last:')
    for country in df.columns:
        last_valid = df[country].last_valid_index()
        if last_valid != None and (not last_valid.startswith('2023')):
            print(country, last_valid)
            
    print()
    print('Done')

In [152]:
class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

In [153]:
countries = 'AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19'

def get_oecd_data(indicator, subject, measure, freq, file_name, 
                  countries_before_subject=False, with_measure=True):
    if freq == 'Q':
        startTime = '1999-Q1'
    else:
        startTime = '1999-01'

    if countries_before_subject:
        countries_and_subject = f'{countries}.{subject}'
    else:
        countries_and_subject = f'{subject}.{countries}'
    
    if with_measure:
        url = f'https://stats.oecd.org/SDMX-JSON/data/{indicator}/{countries_and_subject}.{measure}.{freq}/all?startTime={startTime}&contentType=csv'
    else:
        url = f'https://stats.oecd.org/SDMX-JSON/data/{indicator}/{countries_and_subject}.{freq}/all?startTime={startTime}&contentType=csv'

    print(url)
    res = get_legacy_session().get(url)
    df = pd.read_csv(io.BytesIO(res.content), sep=',')
    df.index = df['TIME']
    
    df_formatted = format_oecd_data(df)
    df_formatted = df_formatted.sort_index()
    check_data_coverage(df_formatted)
    df_formatted.to_csv(f'OECD_{file_name}.csv')
    return df_formatted

## Composite Leading Indicators

In [154]:
indicator = 'MEI_CLI' # Composite Leading Indicators (MEI) 
subject = 'BSCICP03' # OECD Standardised Bussiness Confidence Indicator, Amplitude adjusted (Long term average=100), sa
measure = ''
freq = 'M'
df = get_oecd_data(indicator, subject, measure, freq, with_measure=False, 
                   file_name='Bussiness Confidence Indicator')
df

https://stats.oecd.org/SDMX-JSON/data/MEI_CLI/BSCICP03.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye',
       'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia',
       'Russia'],
      dtype='object')

No data:
Index(['Brasil', 'Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
China 2000-02
India 2000-05
Indonesia 2002-03
Chile 2003-11

Missing last:
Russia 2022-01

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,99.23026,96.94991,96.70270,100.00960,99.90664,99.07822,99.07100,99.88649,100.24430,96.24296,...,NaN,109.57260,97.55158,NaN,101.1002,98.82476,94.73983,90.59782,NaN,99.716455
1999-02,99.53894,97.06934,96.99982,100.14230,99.86806,99.27635,98.89312,99.79908,100.25560,96.50874,...,NaN,107.81200,97.48577,NaN,100.9418,98.68932,94.91841,90.41037,NaN,100.154733
1999-03,99.76861,97.21156,97.54137,100.27130,99.78703,99.32484,98.79162,99.93929,100.24350,96.75919,...,NaN,105.40490,97.52265,NaN,100.9921,98.44651,94.99623,90.56648,NaN,100.682359
1999-04,99.95366,97.36842,98.35950,100.43320,99.87154,99.41689,98.76231,100.18510,100.24270,96.92786,...,NaN,103.56070,97.67751,NaN,101.2359,97.84256,95.71694,91.26705,NaN,101.178524
1999-05,100.16460,97.53513,99.12959,100.65620,100.09060,99.49191,98.80769,100.37760,100.25790,97.17252,...,NaN,102.66100,97.72133,NaN,101.2700,97.44176,96.74817,92.18592,NaN,101.719888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,98.89899,100.29390,99.55035,98.02653,99.91919,100.04330,100.58270,99.76989,100.68350,99.38604,...,99.04103,99.93022,98.49326,95.50326,101.8967,98.97838,98.88395,99.50489,NaN,NaN
2023-05,98.88395,100.34110,99.48969,NaN,99.81200,99.88603,100.21450,99.81442,100.52480,100.02480,...,99.05051,100.08220,98.31992,95.46212,101.7273,99.04419,98.06341,99.06139,NaN,NaN
2023-06,98.86854,100.40470,99.79144,NaN,99.73529,99.77724,99.78830,NaN,100.34810,100.39650,...,99.06405,99.63695,98.26650,95.89169,101.7869,98.88184,97.37190,98.75256,NaN,NaN
2023-07,98.89616,NaN,100.19040,NaN,99.51942,99.67213,99.34898,NaN,100.15180,100.48440,...,NaN,99.08010,98.30476,96.50355,101.8207,98.90376,97.30465,NaN,NaN,NaN


In [155]:
subject = 'CSCICP03' # OECD Standardised Consumer Confidence Indicator, Amplitude adjusted (Long term average=100), sa
df = get_oecd_data(indicator, subject, measure, freq, with_measure=False, 
                   file_name='Consumer Confidence Indicator')
df

https://stats.oecd.org/SDMX-JSON/data/MEI_CLI/CSCICP03.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'France', 'Switzerland',
       'Germany', 'Australia', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Israel', 'Ireland', 'New Zealand',
       'Austria', 'Portugal', 'Eurozone', 'China', 'India', 'Korea',
       'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Canada', 'Norway', 'Brasil', 'Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
Israel 2011-03
India 2012-08
Mexico 2001-04
Indonesia 2001-04
Turkiye 2004-01
Poland 2001-05
Chile 2002-03
Colombia 2001-11

Missing last:
India 2020-09

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,102.16050,99.13457,101.76980,NaN,101.25410,101.18270,100.60430,101.25230,100.96870,99.30415,...,NaN,NaN,NaN,NaN,102.41200,100.58040,96.81049,NaN,NaN,90.73000
1999-02,102.37160,99.22217,102.11630,NaN,101.21350,101.01410,100.49530,101.55030,100.89030,99.39217,...,NaN,NaN,NaN,NaN,102.44690,100.56310,96.45536,NaN,NaN,91.17000
1999-03,102.40730,99.25058,102.41340,NaN,101.09430,100.89680,100.28060,101.57250,100.86780,99.56536,...,NaN,NaN,NaN,NaN,102.52370,100.35150,96.24995,NaN,NaN,91.50000
1999-04,102.40250,99.27444,102.53160,NaN,101.00480,100.83500,100.05560,101.54850,100.96440,99.71352,...,NaN,NaN,NaN,NaN,102.75030,100.00990,96.30946,NaN,NaN,91.72000
1999-05,102.44670,99.32959,102.53680,NaN,101.04590,100.85280,99.90978,101.45660,101.09020,99.87493,...,NaN,NaN,NaN,NaN,102.91550,99.80379,96.39417,NaN,NaN,91.87000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,97.18363,98.13555,96.28214,NaN,97.32153,96.76675,98.86586,97.84637,98.43124,98.21471,...,101.6381,99.50516,99.23917,95.58919,98.90280,98.66436,98.23370,95.16866,NaN,99.31689
2023-05,97.17767,98.38691,97.13757,NaN,97.77294,96.81134,98.99891,97.76232,98.33045,98.59695,...,101.7131,99.88246,100.03850,95.79842,99.50943,98.66811,98.28234,95.77176,NaN,99.61455
2023-06,97.48916,98.52669,97.57776,NaN,98.19633,96.87649,98.98434,97.71120,98.29894,98.85841,...,101.6690,98.92008,100.52920,96.05412,99.99138,98.77574,98.44428,96.42462,NaN,NaN
2023-07,97.95027,98.59281,97.54587,NaN,98.43741,96.96812,98.88734,97.74561,98.36086,98.94753,...,101.5384,96.91089,100.85750,96.21217,100.13840,98.92316,98.81435,96.80355,NaN,NaN


In [156]:
subject = 'LOLITOAA' # Amplitude adjusted (CLI)
df = get_oecd_data(indicator, subject, measure, freq, with_measure=False, 
                   file_name='Composite Leading Indicator')
df

https://stats.oecd.org/SDMX-JSON/data/MEI_CLI/LOLITOAA.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Germany', 'Australia', 'Spain', 'Italy', 'China', 'India', 'Korea',
       'South Africa', 'Mexico', 'Indonesia', 'Turkiye'],
      dtype='object')

No data:
Index(['Switzerland', 'Netherlands', 'Denmark', 'Sweden', 'Finland', 'Belgium',
       'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria', 'Portugal',
       'Eurozone', 'Brasil', 'Saudi Arabia', 'Poland', 'Chile', 'Greece',
       'Hungary', 'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,100.22600,98.08029,99.92517,99.04568,99.87450,NaN,99.32185,100.33960,NaN,NaN,...,93.44679,93.92188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-02,100.43870,98.48177,100.41620,99.32136,99.83424,NaN,99.29385,100.44010,NaN,NaN,...,94.28552,94.92348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-03,100.62990,98.87955,100.91410,99.55447,99.86840,NaN,99.32334,100.52450,NaN,NaN,...,95.19625,95.96504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-04,100.79730,99.25786,101.34050,99.73008,100.00550,NaN,99.41097,100.60160,NaN,NaN,...,96.20809,96.85421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-05,100.94080,99.59679,101.66880,99.85417,100.23400,NaN,99.56577,100.66890,NaN,NaN,...,97.27637,97.49230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,98.89024,100.08740,99.22129,97.52503,98.56210,NaN,99.58736,98.78822,NaN,NaN,...,97.78961,100.28580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05,98.97022,100.13520,99.50819,97.51325,98.53325,NaN,99.52377,98.71284,NaN,NaN,...,97.83102,100.30800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06,99.08180,100.17440,99.72255,97.52862,98.51160,NaN,99.41216,98.64871,NaN,NaN,...,97.91052,100.23180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07,99.20307,100.20680,99.88969,97.56876,98.49852,NaN,99.29068,98.60187,NaN,NaN,...,97.98292,100.09870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Key Short-Term Economic Indicators

In [173]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'IRSTCI01' # Overnight Interbank Rate
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Overnight Interbank Rate')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/IRSTCI01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye',
       'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia',
       'Russia'],
      dtype='object')

No data:
Index(['Brasil', 'Saudi Arabia', 'Argentina'], dtype='object')

Missing first:

Missing last:
Sweden 2020-10
Russia 2022-08

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4.63,0.22526,5.9922,4.991400,3.137000,0.84375,3.137000,4.75,3.137000,3.7699,...,36.43,78.88,15.50,12.94,11.425105,17.9000,9.09,26.98,NaN,60.0
1999-02,4.76,0.17632,5.7693,4.995700,3.118000,1.08036,3.118000,4.75,3.118000,3.5736,...,37.50,77.38,15.50,6.25,10.180684,14.8000,8.00,23.05,NaN,60.0
1999-03,4.81,0.04091,5.5665,4.994300,2.925200,1.00714,2.925200,4.75,2.925200,3.4139,...,37.84,76.32,15.50,6.84,10.147409,15.6000,7.70,22.14,NaN,60.0
1999-04,4.74,0.03143,5.3295,4.777100,2.709100,0.80357,2.709100,4.75,2.709100,3.0343,...,35.19,76.96,15.50,12.81,10.151450,15.3000,7.27,20.72,NaN,60.0
1999-05,4.74,0.03000,5.2689,4.592900,2.551000,0.84052,2.551000,4.75,2.551000,2.8990,...,28.73,76.96,15.50,11.77,10.402900,14.6000,6.91,18.92,NaN,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,4.83,-0.01500,4.1774,4.491718,2.899081,0.90000,2.899081,3.60,2.899081,2.7100,...,5.63,NaN,6.59,11.25,2.899081,17.8314,7.00,13.00,NaN,NaN
2023-05,5.06,-0.05100,4.3528,4.492648,3.046309,0.95000,3.046309,3.83,3.046309,2.9000,...,5.56,NaN,6.74,11.25,3.046309,17.3508,7.00,13.28,NaN,NaN
2023-06,5.08,-0.06600,4.5872,4.683945,3.204590,1.20000,3.204590,4.05,3.204590,3.0900,...,5.61,NaN,6.71,11.25,3.204590,16.0434,7.00,13.28,NaN,NaN
2023-07,5.12,-0.04600,4.9302,4.903890,3.403263,1.20000,3.403263,4.10,3.403263,3.2300,...,5.61,NaN,6.79,11.25,3.403263,15.4338,7.00,13.26,NaN,NaN


In [174]:
subject = 'IR3TIB01' # Short term interest rate
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Short term interest rate')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/IR3TIB01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye',
       'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia',
       'Russia'],
      dtype='object')

No data:
Index(['Brasil', 'Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
Japan 2002-04
India 2011-12

Missing last:
Turkiye 2008-04
Russia 2022-02

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4.89,NaN,5.92171,5.004500,3.1321,NaN,3.1321,4.79,3.1321,3.887500,...,45.50,NaN,12.60,12.75,11.690000,15.8000,8.49,33.23,NaN,43.9
1999-02,4.90,NaN,5.53197,5.033500,3.0934,NaN,3.0934,4.77,3.0934,3.615000,...,38.20,NaN,12.68,10.04,10.664211,15.3000,8.18,30.25,NaN,23.3
1999-03,4.91,NaN,5.42287,5.057391,3.0467,NaN,3.0467,4.81,3.0467,3.459022,...,34.85,NaN,12.84,11.09,10.050455,15.3000,7.58,25.32,NaN,21.6
1999-04,4.88,NaN,5.31773,4.716667,2.6965,NaN,2.6965,4.76,2.6965,3.159773,...,34.09,NaN,12.95,13.59,10.084500,15.4000,7.09,22.08,NaN,20.6
1999-05,4.92,NaN,5.34875,4.615500,2.5790,NaN,2.5790,4.86,2.5790,3.108452,...,31.20,NaN,13.04,11.71,9.945000,14.9000,6.96,19.01,NaN,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,5.03,-0.001,4.52000,4.799086,3.1670,1.45,3.1670,3.66,3.1670,3.281571,...,6.76,NaN,6.90,NaN,3.167000,17.1161,7.18,12.53,NaN,NaN
2023-05,5.15,-0.019,4.73000,4.788520,3.3664,1.45,3.3664,3.90,3.3664,3.396137,...,6.76,NaN,6.90,NaN,3.366400,14.8287,7.18,12.57,NaN,NaN
2023-06,5.22,-0.003,5.09000,5.021803,3.5359,1.45,3.5359,4.25,3.5359,3.562376,...,6.75,NaN,6.90,NaN,3.535900,15.9426,7.15,13.02,NaN,NaN
2023-07,5.35,-0.003,5.49000,5.219326,3.6718,1.72,3.6718,4.30,3.6718,3.720800,...,6.75,NaN,6.76,NaN,3.671800,10.3116,7.11,13.58,NaN,NaN


In [175]:
subject = 'IRLTLT01' # Long-term interest rate
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Long term interest rate')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/IRLTLT01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'South Africa', 'Mexico', 'Indonesia', 'Poland',
       'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Brasil', 'Saudi Arabia', 'Turkiye', 'Argentina'], dtype='object')

Missing first:
China 2014-01
India 2011-12
Korea 2000-10
Mexico 2001-07
Indonesia 2012-01
Poland 2001-01
Chile 2004-07
Czechia 2000-04
Colombia 2003-01

Missing last:
Russia 2018-

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4.72,1.910,4.2868,4.946500,3.7700,2.555,3.702000,5.127895,3.801,4.035,...,NaN,NaN,NaN,NaN,6.30,NaN,NaN,NaN,NaN,96.34
1999-02,5.00,2.117,4.4319,5.124000,3.9290,2.550,3.849000,5.300000,3.932,4.188,...,NaN,NaN,NaN,NaN,6.00,9.27,NaN,NaN,NaN,99.47
1999-03,5.23,1.816,4.6554,5.220435,4.1335,2.604,4.037826,5.526522,4.139,4.432,...,NaN,NaN,NaN,NaN,6.00,9.72,NaN,NaN,NaN,98.67
1999-04,5.18,1.563,4.5920,5.029524,3.9762,2.534,3.850000,5.357500,3.995,4.217,...,NaN,NaN,NaN,NaN,5.90,9.73,NaN,NaN,NaN,110.55
1999-05,5.54,1.334,4.9068,5.359500,4.1600,2.720,4.012105,5.875714,4.193,4.461,...,NaN,NaN,NaN,NaN,5.80,9.72,NaN,NaN,NaN,96.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,3.46,0.385,3.6503,2.909474,2.9200,1.080,2.359444,3.340000,2.719,2.597,...,6.45,NaN,6.07,5.539444,4.23,8.35,4.65,11.89,NaN,NaN
2023-05,3.57,0.430,3.9621,3.045455,2.9400,0.850,2.342273,3.500000,2.728,2.539,...,6.36,NaN,5.93,5.548636,3.98,7.78,4.50,11.11,NaN,NaN
2023-06,3.75,0.395,4.3659,3.338182,2.9300,0.970,2.376364,3.920000,2.750,2.571,...,6.22,NaN,5.91,5.305500,3.70,7.20,4.35,10.15,NaN,NaN
2023-07,3.90,0.595,4.4372,3.452000,3.0400,0.980,2.456667,4.030000,2.840,2.695,...,6.33,NaN,5.56,5.330500,3.85,7.10,4.15,10.22,NaN,NaN


In [176]:
subject = 'LRHUTTTT' # Monthly Unemployment Rate - all persons, s.a.
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Monthly Unemployment')
df.sort_index()

https://stats.oecd.org/SDMX-JSON/data/KEI/LRHUTTTT.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Germany', 'Australia', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'Korea', 'Mexico', 'Turkiye', 'Poland', 'Chile', 'Greece',
       'Hungary', 'Czechia', 'Colombia'],
      dtype='object')

No data:
Index(['Switzerland', 'New Zealand', 'Eurozone', 'China', 'India', 'Brasil',
       'Saudi Arabia', 'South Africa', 'Indonesia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:
Israel 2012-01
Turkiye 2005-01
Colombia 2007-01

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4.3,4.5,6.2,7.9,12.0,NaN,8.9,7.118793,4.6,5.0,...,NaN,NaN,11.5,8.373607,11.4,7.2,7.8,NaN,NaN,NaN
1999-02,4.4,4.6,6.2,7.9,12.0,NaN,8.8,7.115217,4.5,5.0,...,NaN,NaN,12.0,8.810076,11.4,7.2,8.1,NaN,NaN,NaN
1999-03,4.2,4.7,6.2,7.9,11.8,NaN,8.8,7.012740,4.4,5.0,...,NaN,NaN,12.3,9.311584,11.4,7.1,8.6,NaN,NaN,NaN
1999-04,4.3,4.7,6.1,8.2,11.7,NaN,8.7,6.979468,4.4,5.0,...,NaN,NaN,12.7,10.038610,12.1,7.0,8.6,NaN,NaN,NaN
1999-05,4.2,4.7,6.0,7.9,11.5,NaN,8.6,7.025072,4.3,5.0,...,NaN,NaN,12.9,10.649560,12.0,6.9,8.7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,3.4,2.6,4.0,5.0,7.3,NaN,2.9,3.689922,3.4,5.0,...,NaN,10.0,2.8,8.282938,11.6,3.9,2.7,10.288648,NaN,NaN
2023-05,3.7,2.6,4.2,5.2,7.3,NaN,2.9,3.566714,3.5,4.9,...,NaN,9.5,2.8,8.247289,11.2,3.9,2.5,10.096366,NaN,NaN
2023-06,3.6,2.5,4.3,5.4,7.3,NaN,2.9,3.474120,3.5,4.9,...,NaN,9.6,2.8,8.364307,11.0,3.9,2.7,9.365741,NaN,NaN
2023-07,3.5,2.7,NaN,5.5,7.4,NaN,2.9,3.713433,3.6,5.6,...,NaN,9.4,2.8,NaN,10.8,4.0,2.7,9.321362,NaN,NaN


In [177]:
subject = 'LRHUTTTT' # Monthly Unemployment Rate - all persons, s.a.
measure = 'ST' # Level, ratio or index
freq = 'Q' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Quaterly Unemployment')
df.sort_index()

https://stats.oecd.org/SDMX-JSON/data/KEI/LRHUTTTT.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Korea', 'Mexico',
       'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia',
       'Colombia'],
      dtype='object')

No data:
Index(['Eurozone', 'China', 'India', 'Brasil', 'Saudi Arabia', 'South Africa',
       'Indonesia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:
Turkiye 2005-Q1
Colombia 2007-Q1

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,4.300000,4.600000,6.2,7.900000,11.933333,NaN,8.833333,7.082250,4.500000,5.000000,...,NaN,NaN,11.933333,8.810076,11.400000,7.166667,8.166667,NaN,NaN,NaN
1999-Q2,4.266667,4.733333,6.0,7.900000,11.533333,3.099199,8.633333,6.886058,4.333333,4.966667,...,NaN,NaN,12.933333,10.649560,12.033333,6.933333,8.600000,NaN,NaN,NaN
1999-Q3,4.233333,4.700000,5.9,7.500000,11.133333,NaN,8.466667,6.874120,4.166667,5.166667,...,NaN,NaN,13.833333,10.967844,12.033333,6.733333,8.933333,NaN,NaN,NaN
1999-Q4,4.066667,4.633333,5.8,6.966667,10.666667,NaN,8.300000,6.633702,4.033333,5.300000,...,NaN,NaN,14.800000,10.153666,12.333333,6.666667,9.066667,NaN,NaN,NaN
2000-Q1,4.033333,4.833333,5.8,6.866667,10.233333,NaN,8.133333,6.650521,3.866667,4.666667,...,NaN,NaN,15.800000,9.008904,11.566667,6.566667,9.266667,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q3,3.566667,2.566667,3.6,5.133333,7.200000,4.176039,3.100000,3.501735,3.733333,4.533333,...,NaN,10.033333,2.933333,7.827278,12.300000,3.633333,2.366667,10.946416,NaN,NaN
2022-Q4,3.600000,2.533333,3.7,5.100000,7.200000,4.253417,3.066667,3.472797,3.600000,4.666667,...,NaN,10.133333,2.900000,8.411125,11.666667,3.766667,2.400000,11.041762,NaN,NaN
2023-Q1,3.500000,2.600000,3.9,5.000000,7.100000,4.134205,2.966667,3.581322,3.533333,4.800000,...,NaN,9.933333,2.700000,8.721883,11.266667,3.800000,2.533333,10.625934,NaN,NaN
2023-Q2,3.566667,2.566667,4.2,5.200000,7.300000,3.960959,2.900000,3.576919,3.466667,4.933333,...,NaN,9.700000,2.800000,8.247289,11.266667,3.900000,2.633333,9.916919,NaN,NaN


In [178]:
subject = 'CPALTT01' # Consumer Prices - Annual inflation
measure = 'GY' # Growth on the same period of the previous year
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   'Inflation Rate')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/CPALTT01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GY.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'China', 'India', 'Korea', 'Saudi Arabia', 'South Africa',
       'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece',
       'Hungary', 'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'Eurozone', 'Brasil'], dtype='object')

Missing first:
Argentina 2017-12

Missing last:
Japan 2021-06
Russia 2022-03

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,1.670792,0.2,2.0,0.660066,0.244279,0.072915,0.238949,NaN,2.177910,1.703992,...,70.642504,65.901675,6.9,3.594859,3.696482,9.753136,3.587444,17.179262,NaN,96.918604
1999-02,1.605930,-0.1,1.9,0.659341,0.179326,0.288012,0.119190,NaN,2.109408,2.035870,...,53.407429,63.927083,5.6,3.804064,3.689293,9.394904,2.971768,15.379706,NaN,103.242485
1999-03,1.726264,-0.4,2.1,0.987925,0.345269,0.467020,0.238663,NaN,2.147838,2.322206,...,45.416667,63.537401,6.2,4.052286,3.399638,9.274907,2.666667,13.495025,NaN,107.584410
1999-04,2.276923,-0.1,2.0,1.648352,0.421133,0.593769,0.595948,NaN,2.114636,2.364865,...,37.967631,63.854962,6.3,4.033417,2.805966,9.393232,2.511078,11.160355,NaN,113.064572
1999-05,2.088452,-0.4,1.7,1.533406,0.408267,0.575758,0.476190,NaN,2.242028,2.065322,...,30.736258,62.966719,6.4,3.966940,2.372590,8.863680,2.511078,9.998016,NaN,116.711050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,4.930320,NaN,7.8,4.405874,5.884473,2.565343,7.169118,NaN,5.174860,5.282005,...,4.329389,43.684170,14.7,9.907431,3.039219,24.000000,12.710567,12.822871,108.834060,NaN
2023-05,4.047609,NaN,7.9,3.357472,5.119943,2.158705,6.102004,NaN,6.056279,2.928128,...,4.004224,39.585301,13.0,8.726823,2.781513,21.500000,11.060948,12.360256,114.246289,NaN
2023-06,2.969178,NaN,7.3,2.812296,4.531319,1.710877,6.375228,NaN,5.695086,2.464789,...,3.520988,38.213519,11.5,7.560418,1.760478,20.100000,9.703704,12.129618,115.580273,NaN
2023-07,3.177780,NaN,6.4,3.265839,4.288119,1.605105,6.165005,NaN,4.573497,3.133159,...,3.078237,47.831733,10.8,6.475515,2.470822,17.600000,8.771930,11.784920,113.449994,NaN


In [180]:
subject = 'B6BLTT02' # Current Account as a % of GDP
measure = 'ST' # Level, ratio or index
freq = 'Q'
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Current Account')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/B6BLTT02.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Saudi Arabia', 'South Africa', 'Mexico', 'Indonesia',
       'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia',
       'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone', 'Brasil'], dtype='object')

Missing first:
Netherlands 2003-Q2
Denmark 2005-Q1
Belgium 2003-Q1
Ireland 2002-Q1
Saudi Arabia 2006-Q1
Mexico 2006-Q1
Indonesia 2004-Q1
Poland 2004-Q1
Chile 2003-Q1
Greece

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,-2.585638,2.328677,-2.767561,-0.146836,3.757485,2.048831,-1.989932,-5.629524,NaN,NaN,...,NaN,3.256730,NaN,NaN,NaN,-7.830177,-2.908802,NaN,NaN,NaN
1999-Q2,-2.853475,2.460816,-2.203467,-0.252270,3.549500,10.159241,-0.791376,-5.935598,NaN,NaN,...,NaN,-1.226594,NaN,NaN,NaN,-9.332882,-1.883920,NaN,NaN,NaN
1999-Q3,-3.143675,2.575973,-2.207767,0.724169,2.261773,11.742319,-1.359191,-5.656639,NaN,NaN,...,NaN,-0.891263,NaN,NaN,NaN,-8.376171,-0.589354,NaN,NaN,NaN
1999-Q4,-3.300348,2.523838,-1.304133,0.360043,3.720123,10.067650,-1.736559,-4.705191,NaN,NaN,...,NaN,-1.336144,NaN,NaN,NaN,-8.137142,-3.115128,NaN,NaN,NaN
2000-Q1,-3.790054,2.619592,-1.587838,2.086362,1.056238,8.593989,-1.189250,-4.840895,NaN,NaN,...,NaN,-2.774978,NaN,NaN,NaN,-8.108712,-3.747868,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,-3.941371,2.290976,-4.562042,0.684377,-1.282522,6.182092,4.000580,1.973024,6.354128,13.162071,...,0.695421,-3.308629,-3.483302,-12.445438,-10.431840,-7.407750,-6.314773,-5.934337,-1.345756,NaN
2022-Q3,-3.463808,0.744857,-2.032066,-1.399075,-3.192699,14.023609,3.929420,-0.122180,10.298289,16.204049,...,0.575878,-6.925721,-1.994737,-8.704062,-6.976852,-10.365228,-7.221750,-7.161150,-2.059779,NaN
2022-Q4,-3.307890,1.759062,-0.388202,-1.156630,-2.391751,7.806503,4.238211,1.728127,7.614000,12.866754,...,2.195254,-4.887727,-2.154689,-6.013019,-13.418218,-8.013661,-5.790306,-6.168764,0.970018,NaN
2023-Q1,-3.306474,1.915240,-1.662314,-0.452213,-1.160198,9.852435,5.347567,1.941856,10.856231,12.854511,...,1.080092,-6.457548,2.371582,0.610972,1.098207,-3.683799,-0.809308,-3.561610,-2.892637,NaN


In [181]:
subject = 'PRINTO01' # Industrial production
measure = 'GY' # Growth on the same period of the prevoius year
freq = 'M'
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Industrial Production')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/PRINTO01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GY.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'Eurozone', 'India', 'Korea', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'China', 'Brasil', 'Saudi Arabia',
       'South Africa', 'Mexico', 'Indonesia', 'Argentina'],
      dtype='object')

Missing first:
Switzerland 2011-10

Missing last:
Russia 2021-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,3.638185,-6.206206,2.144470,5.455559,1.415701,NaN,0.400534,NaN,1.085776,1.806589,...,NaN,-5.990783,-4.123498,-5.357759,5.307198,13.445378,-8.702718,-14.437933,NaN,-2.243915
1999-02,4.058844,-3.699897,0.783875,4.073814,0.412939,NaN,-1.204819,NaN,3.193833,0.858369,...,NaN,-5.517241,-5.170261,-4.866265,2.622234,5.548038,-7.656236,-17.851952,NaN,-9.360208
1999-03,4.243102,-0.315126,1.339286,2.673290,1.542135,NaN,-1.849406,NaN,3.837719,1.052632,...,NaN,-4.977376,1.832251,-3.455285,-5.578570,8.502024,-9.609728,-22.563792,NaN,-5.861282
1999-04,4.122832,-1.789474,1.230425,3.755275,1.031130,NaN,-0.266667,NaN,0.867679,0.333704,...,NaN,-4.387991,-0.036993,-5.374827,-2.161938,8.010681,-7.047759,-19.893567,NaN,-5.771179
1999-05,4.075301,0.530223,3.636364,4.395719,0.174588,NaN,-0.132626,NaN,-0.214823,3.543743,...,NaN,-3.386005,3.220337,-3.431228,3.566775,4.427083,-1.845239,-20.107370,NaN,-1.137348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,0.323069,0.190476,-1.313321,0.719543,1.514995,NaN,2.531646,NaN,-11.695376,10.162917,...,NaN,-0.889802,-3.405370,-1.012847,4.506066,-5.693665,1.687389,-4.755581,NaN,NaN
2023-05,0.009243,2.300000,-2.061856,0.605288,2.238193,NaN,1.668405,NaN,-8.820799,6.495468,...,NaN,0.410397,-2.316347,-5.392686,1.684920,-4.313099,0.612423,-3.345041,NaN,NaN
2023-06,-0.310655,-0.189573,0.757576,-0.393338,-0.262892,NaN,0.000000,NaN,-6.308851,11.483967,...,NaN,0.202293,-0.935204,-2.218562,-3.640777,-5.859066,0.344234,-4.805130,NaN,NaN
2023-07,-0.039561,-2.358491,0.475285,NaN,2.674515,NaN,-1.352758,NaN,-6.454630,4.217791,...,NaN,6.931408,-2.200000,-2.002862,-2.168675,-2.627389,-2.321582,NaN,NaN,NaN


In [182]:
subject = 'NAEXKP02' # Private Consumption
measure = 'GY' # Growth on the same period of the prevoius year
freq = 'Q'
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Private Consumption')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/NAEXKP02.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GY.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'India', 'Korea',
       'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone', 'China', 'Brasil', 'Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
India 2012-Q2
Colombia 2006-Q1
Russia 2004-Q1

Missing last:
Russia 2021-Q3

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,5.651322,0.861539,3.456293,3.576515,3.023039,2.191664,2.650972,6.490941,6.640931,1.752536,...,2.188967,-0.725987,4.109588,-3.043256,3.051948,6.109058,3.365542,NaN,NaN,NaN
1999-Q2,5.413038,1.039537,4.753388,3.362945,2.821045,2.099397,3.266317,5.495034,6.322037,-0.222539,...,3.892209,3.527555,6.221875,-4.940129,2.276205,6.912400,3.523816,NaN,NaN,NaN
1999-Q3,5.225906,1.043997,4.130955,4.047948,4.000820,2.025536,2.008873,4.613198,5.689979,-0.513521,...,7.656950,-1.874710,4.484617,-0.730095,3.114917,7.079424,3.645058,NaN,NaN,NaN
1999-Q4,5.244597,1.470688,4.878311,4.743512,3.845444,2.284799,1.993951,5.055635,5.013944,-1.447840,...,4.899022,-0.539694,6.698273,4.840225,1.424292,6.508055,1.928596,NaN,NaN,NaN
2000-Q1,5.749823,1.647361,5.156592,4.343575,4.771746,2.159169,1.646271,3.477669,4.753546,-0.019015,...,0.806698,3.610387,4.363981,5.015551,2.407057,4.243081,1.268848,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,2.365147,2.559188,4.826089,7.926034,4.760390,4.331350,5.264206,5.295827,7.001637,-1.462915,...,4.880922,21.010991,6.386004,7.478210,8.829624,8.034160,-0.130161,14.910579,NaN,NaN
2022-Q3,2.192283,3.746768,1.633502,3.138237,-0.030272,2.652657,1.605397,11.547891,2.890485,-3.252893,...,5.212993,18.131735,2.024316,-3.414986,5.481078,4.328660,-4.543280,9.129451,NaN,NaN
2022-Q4,1.668982,0.955686,1.648990,3.039665,-0.883391,2.996229,0.512529,5.198754,4.237358,-4.918704,...,5.059106,15.863127,-1.412260,-4.409712,3.957130,1.234183,-5.258664,3.922484,NaN,NaN
2023-Q1,2.366445,2.617380,0.257922,3.756974,0.410875,3.119141,-0.368062,3.562526,3.174298,-1.067334,...,5.027742,16.847549,-2.562805,-7.135776,2.496861,-2.744051,-5.322971,3.055144,NaN,NaN


## Quaterly National Accounts

In [183]:
indicator = 'QNA' # Quaterly National Accounts
subject = 'B1_GE' # Gross domestic product - expenditure approach
measure = 'GYSA' # Growth rate based on seasonally adjusted volume data, percentage change on the same quarter the previous year
freq = 'Q' # Quaterly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Quaterly GDP')
df

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.B1_GE.GYSA.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Saudi Arabia', 'South Africa', 'Mexico', 'Indonesia',
       'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia',
       'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone', 'Brasil'], dtype='object')

Missing first:
Saudi Arabia 2011-Q1
Colombia 2006-Q1
Russia 2004-Q1

Missing last:
Russia 2021-Q3

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,4.821858,-0.840503,2.193433,4.194104,2.784202,1.030432,0.793797,5.173796,4.213417,2.311058,...,-6.407874,-4.090021,2.100109,-2.237507,2.361704,2.520821,0.395327,NaN,-2.176492,NaN
1999-Q2,4.727532,-0.033774,2.063279,5.009661,2.617900,0.571309,1.220621,4.541221,4.624423,3.621945,...,1.497007,-3.864924,4.224077,-3.605963,2.861976,2.067932,0.575203,NaN,-5.140582,NaN
1999-Q3,4.800205,0.330989,3.467173,5.690714,3.529290,1.175652,1.859483,3.776954,5.167647,2.381317,...,2.921313,-4.137498,5.298836,-0.906906,2.896443,3.041622,1.473288,NaN,-5.124917,NaN
1999-Q4,4.827627,-0.435648,4.314616,5.738707,4.179188,3.735687,3.124281,3.962759,5.680132,3.481499,...,5.991807,-1.017627,6.529382,5.105253,3.724339,4.188502,2.542947,NaN,-1.014421,NaN
2000-Q1,4.225956,2.667340,4.774235,5.516373,4.357079,4.320257,3.399247,3.623640,4.594874,3.552325,...,3.483446,2.265206,6.377840,6.077906,3.330422,5.153192,3.879361,NaN,-0.106447,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,1.796042,1.434724,3.841392,4.719747,3.875825,3.288302,1.555184,3.092582,5.161248,3.151989,...,5.288928,6.987693,5.977937,5.499798,7.123086,6.229507,3.359600,12.151756,7.688863,NaN
2022-Q3,1.942138,1.520914,1.976076,3.818950,1.180414,1.377187,1.245724,5.994062,2.994482,2.284758,...,6.111392,3.782324,5.006772,0.007175,4.050557,3.744120,1.350071,7.332106,5.342538,NaN
2022-Q4,0.881278,0.461850,0.573879,2.074066,0.719408,1.153463,0.824963,2.696466,3.061577,0.372316,...,5.157855,3.354782,0.990492,-2.137102,4.754101,0.683511,0.120022,1.970428,1.102267,NaN
2023-Q1,1.800193,1.849551,0.231021,2.079295,0.826340,1.443426,-0.268399,2.444471,2.088185,2.029691,...,4.940213,3.005379,-0.618343,-0.957612,1.968942,-1.195203,-0.422889,2.998554,1.225916,NaN
